In [1]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
#Importing the libraries
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
pandas version: 0.25.1
magellan version: 0.3.3


In [3]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = './data/restaurants.csv'
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_A, key='id')
print(type(A))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


<class 'pandas.core.frame.DataFrame'>


In [4]:
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 864
Number of tuples in B: 864
Number of tuples in A X B (i.e the cartesian product): 746496


In [5]:
# Displaying first two entries from the first music dataset
A.head(2)

,address,city,id,name,phone,type
0,435 s la cienega blv,los angeles,1,arnie morton s of chicago,310 246 1501,american
1,435 s la cienega blvd,los angeles,2,arnie morton s of chicago,310 246 1501,steakhouses


In [6]:
# Displaying first two entries from the second music dataset
B.head(2)

,address,city,id,name,phone,type
0,435 s la cienega blv,los angeles,1,arnie morton s of chicago,310 246 1501,american
1,435 s la cienega blvd,los angeles,2,arnie morton s of chicago,310 246 1501,steakhouses


In [7]:
# Display the keys of the input tables
print(em.get_key(A), em.get_key(B))
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 1000, 1, show_progress=False)
print("Lengths after downsampling-")
print(len(A1), len(B1))

Size of table B is less than b_size parameter - using entire table B


id id
Lengths after downsampling-
854 864


In [8]:
colA = ["address","city","id","name","phone","type"]

Feature Generation

In [20]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [21]:
# List the names of the features generated
feature_table['feature_name']

0         address_address_jac_qgm_3_qgm_3
1     address_address_cos_dlm_dc0_dlm_dc0
2     address_address_jac_dlm_dc0_dlm_dc0
3                     address_address_mel
4                address_address_lev_dist
5                 address_address_lev_sim
6                     address_address_nmw
7                      address_address_sw
8               city_city_jac_qgm_3_qgm_3
9           city_city_cos_dlm_dc0_dlm_dc0
10          city_city_jac_dlm_dc0_dlm_dc0
11                          city_city_mel
12                     city_city_lev_dist
13                      city_city_lev_sim
14                          city_city_nmw
15                           city_city_sw
16                              id_id_exm
17                              id_id_anm
18                         id_id_lev_dist
19                          id_id_lev_sim
20              name_name_jac_qgm_3_qgm_3
21          name_name_cos_dlm_dc0_dlm_dc0
22          name_name_jac_dlm_dc0_dlm_dc0
23                          name_n

In [22]:
lit = []

for i in range(44):
    if(i%8!=5):
        lit.append(i)
        
# feature_table = feature_table.drop([28,29,30,36,37,38,39,40,41,42,43],axis=0) 

feature_table = feature_table.drop(lit,axis=0) 

# em.add_blackbox_feature(feature_table, 'time_time_feature', time_feature)
# em.add_blackbox_feature(feature_table, 'released_released_feature', released_feature)
# em.add_blackbox_feature(feature_table, 'price_price_feature', price_feature)
# em.add_blackbox_feature(feature_table, 'copyright_copyright_feature', copyright_feature)

In [23]:
# List the names of the features generated
feature_table['feature_name']

5             address_address_lev_sim
13                  city_city_lev_sim
21      name_name_cos_dlm_dc0_dlm_dc0
29    phone_phone_cos_dlm_dc0_dlm_dc0
37      type_type_cos_dlm_dc0_dlm_dc0
Name: feature_name, dtype: object

In [24]:
# Load the pre-labeled data
path_G = './data/labeled_res_data.csv'

# path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
G = em.read_csv_metadata(path_G, key="_id",
                         ltable=A, rtable=B, 
                         fk_ltable='ltable.id', fk_rtable='rtable.id')
print(len(G))

Metadata file is not present in the given path; proceeding to read the csv file.


746496


In [25]:
H = em.extract_feature_vecs(G, feature_table=feature_table)
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:12:37


,_id,ltable.id,rtable.id,address_address_lev_sim,city_city_lev_sim,name_name_cos_dlm_dc0_dlm_dc0,phone_phone_cos_dlm_dc0_dlm_dc0,type_type_cos_dlm_dc0_dlm_dc0
0,1,1,1,1.000000,1.000000,1.000000,1.000000,1.0
1,2,1,2,0.954545,1.000000,1.000000,1.000000,0.0
2,3,1,3,0.333333,0.000000,0.258199,0.000000,1.0
3,4,1,4,0.333333,0.000000,0.258199,0.000000,0.0
4,5,1,5,0.285714,0.181818,0.000000,0.333333,0.0
...,...,...,...,...,...,...,...,...
746491,746492,864,860,0.461538,1.000000,0.000000,0.333333,0.0
746492,746493,864,861,0.352941,1.000000,0.000000,0.333333,0.0
746493,746494,864,862,0.307692,1.000000,0.000000,0.333333,0.0
746494,746495,864,863,0.538462,1.000000,0.000000,0.333333,0.0


In [34]:
import pickle

dbfile = open('f_vec', 'ab') 
pickle.dump(H, dbfile)                      
dbfile.close()

In [36]:
import pickle

dbfile = open('f_vec', 'rb')      
H = pickle.load(dbfile) 
H

,_id,ltable.id,rtable.id,address_address_lev_sim,city_city_lev_sim,name_name_cos_dlm_dc0_dlm_dc0,phone_phone_cos_dlm_dc0_dlm_dc0,type_type_cos_dlm_dc0_dlm_dc0
0,1,1,1,1.000000,1.000000,1.000000,1.000000,1.0
1,2,1,2,0.954545,1.000000,1.000000,1.000000,0.0
2,3,1,3,0.333333,0.000000,0.258199,0.000000,1.0
3,4,1,4,0.333333,0.000000,0.258199,0.000000,0.0
4,5,1,5,0.285714,0.181818,0.000000,0.333333,0.0
...,...,...,...,...,...,...,...,...
746491,746492,864,860,0.461538,1.000000,0.000000,0.333333,0.0
746492,746493,864,861,0.352941,1.000000,0.000000,0.333333,0.0
746493,746494,864,862,0.307692,1.000000,0.000000,0.333333,0.0
746494,746495,864,863,0.538462,1.000000,0.000000,0.333333,0.0


In [37]:
n = len(feature_table)
print(n)

w = [1/n]*n

print(w)

5
[0.2, 0.2, 0.2, 0.2, 0.2]
